# パラメータ実験用のディレクトリを追加する

このタスクでは、データセットの構造として、パラメータ実験用のデータ構成( `for_parameters` )を指定した研究者を対象に、  
実験パッケージ内にパラメータ実験用のディレクトリを追加します。

![task_paramexp_name](./images/task_paramexp_name.png)

実験フロートップページに戻る場合は[こちら](../experiment.ipynb)。新規タブで開きます。  

**データセットの構造をコード用の実験パッケ―ジ( `with_code` )を指定した場合、このタスクは不要です。**<br>
**データセットの構造が不明の場合は以下の「タスクの実行対象確認」で確認できます。**

#### タスクの実行対象確認
データセットの構造が不明の場合は以下のセルを実行すること、本タスクの対象データセット構造であるか確認できます。

In [ ]:
import os
import json
from IPython.display import display, HTML, Javascript

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

# DS構成のスキーム名をdmp.jsonから取得する
assigned_values = utils.fetch_gin_monitoring_assigned_values()
scheme_name = assigned_values['datasetStructure']

if scheme_name == 'for_parameters':
    print('本タスクの対象です。パラメータ実験用のディレクトリを追加したい場合は以下の処理に進んでください。')
if scheme_name != 'for_parameters':
    print("本タスクの対象外です。以下のリンクから実験フロートップページに戻ってください。")
    display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))

display(Javascript('IPython.notebook.save_checkpoint();'))

## 0. 研究リポジトリ名・実験パッケージ名を確認する  
以下のセルを実行すると、この実験実行環境で操作する実験パッケージの名前と、実験パッケージの存在する研究リポジトリ名を確認できます。  

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

%store -r
if 'EXPERIMENT_TITLE' not in locals().keys() : EXPERIMENT_TITLE = '-'
utils.show_name('blue', EXPERIMENT_TITLE)

##  1. 『パラメータ実験用ディレクトリの追加』のためのセクションを有効化する

本セクションでは、「パラメータ実験用のディレクトリを追加する」タスクの以降のセクションの実行可能状態を有効化します。<br>
**パラメータ実験用のディレクトリを追加したい場合は必ず実行してください。**<br>

パラメータ実験用のデータ構成を利用していない場合は、有効化されず、以降の`パラメータ実験用のディレクトリの追加`に必要なセクション(2.~3.)を実行しても無効となります。

In [ ]:
import os
import json
from IPython.display import display, HTML, Javascript

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

# DS構成のスキーム名をdmp.jsonから取得する
assigned_values = utils.fetch_gin_monitoring_assigned_values()
scheme_name = assigned_values['datasetStructure']

is_activation = False

if scheme_name == 'for_parameters':
    is_activation = True
    print('以降のセクションが有効されました。パラメータ実験用のディレクトリを追加したい場合は以下の処理に進んでください。')
if scheme_name != 'for_parameters':
    is_activation = False
    print("本タスクの対象外です。以下のリンクから実験フロートップページに戻ってください。")
    display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))

display(Javascript('IPython.notebook.save_checkpoint();'))

## 2. パラメータ実験用のディレクトリを追加する

### 2-1. パラメータ実験用のディレクトリを用意する

In [ ]:
import os

# このコンテナで扱う実験パッケージのパスを作成する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE
try:
    if is_activation:
        # /home/jovyan 配下に移動する
        os.chdir(os.environ['HOME'])

        # ディレクトリを追加する
        !cp -r ~/WORKFLOWS/PACKAGE/scheme/$scheme_name/parameter $experiment_path
        print("次にお進みください。")
    else:
        print("本タスクの対象外です。以下のリンクから実験フロートップページに戻ってください。")
        display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))
except NameError as e :
    print("『1. 『パラメータ実験用ディレクトリの追加』のためのセクションを有効化する』セクションが実行されていません。")


### 2-2. パラメータ実験名を決定する

パラメータ実験名を入力して、実験データ格納用のディレクトリを作成します。

パラメータ実験名としては、実験パッケージ内で同名のパラメータ実験名を使用することはできません。  
また、「parameter」というパラメータ実験名も使用することができません。パラメータ名など分かりやすい名前をご記入ください。  

**入力値に誤りがある場合は、以下を参照して対応してください。**
- 手順2-3以降が未実行である場合は、当セルを再度実行して訂正してください。  
- 手順2-3が実行済みである場合は、当セルを再度実行しても訂正ができません。以降の全てのセルを最後まで実行した後に、当タスクを最初のセルから再度実行して必要なパラメータ実験を用意してください。


In [ ]:
import os
import re
import shutil
from IPython.display import clear_output

try:
    if is_activation:
        if 'dir_creation_completed' in locals() and dir_creation_completed==True:
            print("手順2-3が実行済みですので、手順2-2を実行いただけません。パラメータ実験名の入力を誤った場合は、以降の全てのセルを最後まで実行した後に、当タスクを最初のセルから再度実行して必要なパラメータ実験を用意してください。")
        else:
            # GINサーバのものに合わせたバリデーションルールを設定
            validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

            print('作成したいパラメータ実験名を半角英数字で入力してください。')
            while True:
                print("入力完了後、Enterキーを押下してください。")
                paramexp_title = input("パラメータ実験名：")
                if validation.fullmatch(paramexp_title):
                    break
                else:
                    clear_output()
                    print('パラメータ実験名は英数字、および"-", "_", "."のみで入力してください。')
            clear_output()
            print("作成したパラメータ実験名：", paramexp_title)
            print('このパラメータ実験名で処理を進めます。')
    else:
        print("本タスクの対象外です。以下のリンクから実験フロートップページに戻ってください。")
        display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))
except NameError as e :
    print("『1. 『パラメータ実験用ディレクトリの追加』のためのセクションを有効化する』セクションが実行されていません。")

### 2-3. 入力したパラメータ実験名でディレクトリを用意する

In [ ]:
from IPython.display import clear_output
try:
    if is_activation:
        if 'dir_creation_completed' in locals() and dir_creation_completed==True:
            print("手順2-3はすでに実行済みです。次の処理にお進みください。")
        else:
            # 該当実験パッケージを特定させるため、環境変数EXPERIMENT_TITLEに実験パッケージ名を設定
            PARAMEXP_TITLE = paramexp_title
            %store PARAMEXP_TITLE
            clear_output()

            # 実験パッケージの直下に移動
            os.chdir(experiment_path)

            # parameterディレクトリをユーザが指定したパラメータ実験名に変更
            shutil.move('parameter', paramexp_title)
            dir_creation_completed = True
            print('入力したパラメータ実験名「' + paramexp_title + '」でディレクトリを用意しました。')
    else:
        print("本タスクの対象外です。以下のリンクから実験フロートップページに戻ってください。")
        display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))
except NameError as e :
    print("『1. 『パラメータ実験用ディレクトリの追加』のためのセクションを有効化する』セクションが実行されていません。")

## 3. 本タスクの実行結果をGIN-forkに同期する

ここまでの内容を保存し、GIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob
try:
    if is_activation:
        os.chdir(experiment_path)

        #**************************************************#
        #* Generate a list of folder paths to be managed by Git-annex. #
        #**************************************************#
        dirlist=[]
        filelist=[]
        annexed_save_path=[]

        # Recursively search under the experimental package to obtain a list of absolute directory paths.
        for root, dirs, files in os.walk(top=experiment_path):
            for dir in dirs:
                dirPath = os.path.join(root, dir)
                dirlist.append( dirPath )   

        # Add directory paths containing the string "output_data" that are not included under input_data to annexed_save_path.
        output_data_path = [ s for s in dirlist if 'output_data' in s ]
        for output_data in output_data_path:  
            if  "input_data" not in output_data:
                annexed_save_path.append( output_data )

        # Add the input_data directory to annexed_save_path.
        annexed_save_path.append( experiment_path + '/input_data'  )

        # Generate a list of file paths to which metadata is to be assigned.
        gitannex_files = []
        for path in annexed_save_path:
            gitannex_files += [p for p in glob.glob(path+'/**', recursive=True)
                    if os.path.isfile(p)]

        #********************************************************#
        #* Generate a list of directory paths and file paths to be managed by Git. #
        #********************************************************#
        # Obtain a list of directories and files directly under the experimental package.
        files = os.listdir()

        # Delete Git-annex managed directories (input_data and output_data) from the retrieved list.
        dirs = [f for f in files if os.path.isdir(f)]

        for dirname in dirs:
            if dirname == 'input_data' :
                dirs.remove('input_data')

            if dirname == 'output_data' :
                dirs.remove('output_data')

        for dirname in dirs:
            if dirname != 'ci' and dirname != 'source':
                full_param_dir = '{}/{}/params'.format(experiment_path,dirname)
                if os.path.isdir(full_param_dir):
                    dirs.remove(dirname)
                    ex_param_path = '{}/{}'.format(experiment_path, dirname)
                    ex_param_path_childs = os.listdir(ex_param_path)
                    for ex_param_path_child in ex_param_path_childs:
                        if ex_param_path_child != 'output_data':
                            dirs.append('{}/{}'.format(dirname,ex_param_path_child))
            
        # Obtain files directly under the experimental package.
        files = [f for f in files if os.path.isfile(f)]

        # Generate a list of folder paths and file paths to be managed by Git.
        files.extend(dirs)
        save_path = []
        for file in files:
            save_path.append(experiment_path + '/' + file)
        print("次にお進みください。")
    else:
        print("本タスクの対象外です。以下のリンクから実験フロートップページに戻ってください。")
        display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))
except NameError as e :
    print("『1. 『パラメータ実験用ディレクトリの追加』のためのセクションを有効化する』セクションが実行されていません。")

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[GINへの同期の失敗を解消する。](../conflict_helper.ipynb)を参照して、競合を解消してください。


In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
try:
    if is_activation:
        os.chdir(os.environ['HOME'])
        get_path = 'experiments/{}'.format(EXPERIMENT_TITLE)
        # save this notebook
        save_path.append('WORKFLOWS/EX-WORKFLOWS/prepare_parameter_experiment.ipynb')
        is_ok = utils.syncs_with_repo(git_path=save_path, gitannex_path=annexed_save_path, gitannex_files=gitannex_files, message=EXPERIMENT_TITLE + '_パラメータ実験用意', get_paths=[get_path])
        del dir_creation_completed
    else:
        print("本タスクの対象外です。以下のリンクから実験フロートップページに戻ってください。")
        display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))
except NameError as e :
    print("『1. 『パラメータ実験用ディレクトリの追加』のためのセクションを有効化する』セクションが実行されていません。")

## 4. 実験フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして実験フロートップページに戻ってください。  

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))